In [0]:
import pyspark

#Creamos una sesion de spark
from pyspark.sql import SparkSession

#Creamos un objeto de spark session
spark = SparkSession.builder.appName("pysparkdf").getOrCreate()

In [0]:
# Lecura de fuentes de datos
df21 = spark.read.option("header", "true").csv("dbfs:/FileStore/tables/uno/world_happiness_report_2021.csv")
df = spark.read.option("header", "true").csv("dbfs:/FileStore/tables/uno/world_happiness_report.csv")


In [0]:
## 1.- El país de 2021 más feliz (Ladder score)
from pyspark.sql.functions import max
max_ladder_score_row = df21.orderBy("Ladder score", ascending=False).first()
print("País con el máximo Ladder score:")
print("Country name:", max_ladder_score_row["Country name"])
print("Ladder score:", max_ladder_score_row["Ladder score"])
#display(max_ladder_score_row["Country name"])


País con el máximo Ladder score:
Country name: Finland
Ladder score: 7.842


In [0]:
## 2.- Máxima felicidad por Indicador Regional
from pyspark.sql.functions import max
grouped = df21.groupBy("Regional indicator").agg(max("Ladder score").alias("max_ladder_score"))
ordered = grouped.orderBy("max_ladder_score", ascending=False)
display(ordered)

Regional indicator,max_ladder_score
Western Europe,7.842
North America and ANZ,7.277
Middle East and North Africa,7.157
Latin America and Caribbean,7.069
Central and Eastern Europe,6.965
East Asia,6.584
Southeast Asia,6.377
Commonwealth of Independent States,6.179
Sub-Saharan Africa,6.049
South Asia,5.269


In [0]:
## 3.- País más feliz en cada año

from pyspark.sql import functions as F
from pyspark.sql.window import Window

window_spec = Window.partitionBy("year").orderBy(F.col("Life Ladder").desc())
grouped_with_rank = df.withColumn("rank", F.row_number().over(window_spec))
result = grouped_with_rank.filter("rank = 1").drop("rank")

display(result)

Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
Denmark,2005,8.019,10.851,0.972,69.600,0.971,null,0.237,0.860,0.154
Finland,2006,7.672,10.745,0.965,69.760,0.969,-0.005,0.132,0.722,0.172
Denmark,2007,7.834,10.891,0.954,69.920,0.932,0.240,0.206,0.828,0.194
Denmark,2008,7.971,10.880,0.954,70.080,0.970,0.272,0.248,0.757,0.163
Denmark,2009,7.683,10.824,0.939,70.240,0.949,0.264,0.206,0.749,0.234
Denmark,2010,7.771,10.839,0.975,70.400,0.944,0.242,0.175,0.785,0.155
Denmark,2011,7.788,10.848,0.962,70.620,0.935,0.298,0.220,0.769,0.175
Switzerland,2012,7.776,11.079,0.947,72.780,0.945,0.139,0.323,0.859,0.176
Canada,2013,7.594,10.757,0.936,72.680,0.916,0.316,0.406,0.851,0.263
Denmark,2014,7.508,10.862,0.956,71.280,0.942,0.118,0.237,0.832,0.233
